In [1]:

import sys
from pathlib import Path
parent_dir = Path.cwd().parent
sys.path.append(str(parent_dir))

import numpy as np
import torch

from datasets import load_dataset
from src.dataset import Dataset
from src.model_dump import Dump
from src.model_phase import PhraseModel
from src.model_query import QueryModel

raw_datasets = load_dataset("squad")

/home/ubuntu/roseltorg_devs/research/developers/Artem-folder/DensePhrasesOld/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
dataset = Dataset()
dataset.load_ds(raw_datasets, size=500)

100%|██████████| 500/500 [00:00<00:00, 1607.26it/s]


In [4]:
   
phrasemodel = PhraseModel(hidden_dim=264, ds=dataset, model_checkpoint='SpanBERT/spanbert-base-cased')
optimizer = torch.optim.Adam(phrasemodel.parameters(), lr=0.01)
L = len(dataset.contexts)
indices = list(range(L))
np.random.shuffle(indices)
batch_size = 10
inputs_batched = [indices[i*batch_size:(i+1)*batch_size]
                          for i in range(L //batch_size+1)]

Some weights of BertModel were not initialized from the model checkpoint at SpanBERT/spanbert-base-cased and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BertModel were not initialized from the model checkpoint at SpanBERT/spanbert-base-cased and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BertModel were not initialized from the model checkpoint at SpanBERT/spanbert-base-cased and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
# phrasemodel(inputs)

In [5]:
num_epochs = 10
for epoch in tqdm(range(num_epochs)):
    # Forward pass
    for inputs in inputs_batched:
        loss = phrasemodel(inputs)
        

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(loss.item())

  0%|          | 0/10 [00:00<?, ?it/s]/tmp/ipykernel_6807/550987792.py:33: UserWarning: The use of `x.T` on tensors of dimension other than 2 to reverse their shape is deprecated and it will throw an error in a future release. Consider `x.mT` to transpose batches of matrices or `x.permute(*torch.arange(x.ndim - 1, -1, -1))` to reverse the dimensions of a tensor. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3637.)
  z_start = torch.matmul(last_hidden_state[n], q_start[n].T).reshape(context_num_tokens)
/home/ubuntu/roseltorg_devs/research/developers/Artem-folder/DensePhrases/.venv/lib/python3.10/site-packages/torch/nn/modules/module.py:1511: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)
 10%|█         | 1/10 [14:14<2:08:06, 854.01s/it]

-0.01195745449513197


 20%|██        | 2/10 [29:24<1:58:16, 887.10s/it]

-0.01134384237229824


 30%|███       | 3/10 [42:36<1:38:25, 843.64s/it]

...

-0.011356784962117672


 90%|█████████ | 9/10 [1:52:45<11:42, 702.73s/it]

-0.011314444243907928


100%|██████████| 10/10 [2:04:12<00:00, 745.26s/it]

-0.009065413847565651


In [5]:
dump = Dump(dataset, phrasemodel.model, 
              phrasemodel.model_1, phrasemodel.model_2,
              phrasemodel.tokenizer, hidden_dim=768)



In [6]:
dump.create_dump()

Creating Phrase dump:  50%|█████     | 30/60 [00:07<00:10,  2.86it/s]

In [ ]:
dump.token_w_id2context_id[0]

0

In [6]:
dump.predict(0, k=100, verbose=True)

Q: To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?
C: Architecturally, the school has a Catholic character. Atop the Main Building's gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.
73 73

73 73

73 0
73 33
73 33
73 73

73 26
73 26
73 81
73 81
73 70
73 70
73 79
73 79
73 19
73 19
73 60
73 60
73 31
73 71
73 71
73 57
73 57
73 94
73 94
73 86
73 86
73 83
73 83
73 100
73 100
73 66
73 66
73 22
73 22
73 31
73 31


('[CLS] beyonce embarked on the mrs. carter show world tour on april 15 in belgrade, serbia ; the tour included 132 dates that ran through to march 2014. it became the most successful tour of her career and one of the most - successful tours of all time. in may, beyonce\'s cover of amy winehouse\'s " back to black " with andre 3000 on the great gatsby soundtrack was released. she was also honorary chair of the 2013 met gala. beyonce voiced queen tara in the 3d cgi animated film, epic, released by 20th century fox on may 24, and recorded an original song for the film, " rise up ", co - written with sia.',
 21.617016)

In [13]:
for i in range(100):
    output = model.predict(i, k=100, verbose=True)
    if output[0]:
        print(output)

Q: To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?
C: Architecturally, the school has a Catholic character. Atop the Main Building's gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.
Q: What is in front of the Notre Dame Main Building?
C: Architecturally, the school has a Catholic character. Atop the Main Building's gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building 

In [241]:
loss.requires_grad

False

In [239]:
optimizer.zero_grad()
loss.backward()
optimizer.step()

RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn